In [ ]:
# Python script for data validation, duplicate check, and concatenation of 'non-visible' crabs of a raw dataset including instantaneous observations of Afruca tangeri in two crabitats (110 tank & tub) at the Sainsbury Wellcome Centre for Neural Circuits and Animal Behaviour throughout 2022-2023.
# Composed by Sanna Titus | sannatitus@gmail.com | 1 October 2024

import pandas as pd
import os
import datetime

####################################################################
# DATA IMPORT 
####################################################################
file_path = 'https://raw.githubusercontent.com/sannatitus/Afruca-tangeri_captive-behaviour/main/Data/instantaneous-behaviour-raw-data.xlsx' # note the identical instantaneous-behaviour-raw-data.xlsx and instantaneous-behaviour-raw-data.csv have already been through this pipeline, and thus will not report any validation issues nor concatenate NV crabs.  
df = pd.read_excel(file_path, sheet_name='data', engine='openpyxl') # this engine is necessary for xlsb

####################################################################
# DATA VALIDATION
####################################################################

# Check that each crab ID has exactly 30 rows in each unique observation window
def validate_crab_id_rows(df):
    # Unique observation windows = group by video file, tide category, and crab ID
    grouped = df.groupby(['video file', 'tide category', 'crab ID'])

    validation_issues = []

    for (video_file, tide_category, crab_id), group in grouped:
        count = len(group)
        if count != 30:
            validation_issues.append({
                'video file': video_file,
                'tide category': tide_category,
                'crab ID': crab_id,
                'row count': count
            })

    return validation_issues

validation_results = validate_crab_id_rows(df_with_tide_categories_and_types)

if validation_results:
    validation_summary_df = pd.DataFrame(validation_results)
    
    validation_summary_path = f"{file_name}_cleaned_validation.xlsx"
    
    validation_summary_df.to_excel(validation_summary_path, index=False)
    print(f"Validation summary saved to: {validation_summary_path}")
    for issue in validation_results:
        print(f"Crab ID: {issue['crab ID']} in {issue['video file']} - {issue['tide category']} has {issue['row count']} rows.")
else:
    print("All crab IDs have the correct number of rows (30) in each observation window.")

####################################################################
# 'Non-Visible' (NV) CRAB CALCULATION 
####################################################################
# When aquiring this dataset, the number of crabs present in each observation window was always recorded. Using population (and invidual sex) data from husbandry records, we can calculate the number of NV crabs (and their sexes) in each observation window. The number of NV crabs is equal to the difference between the number of crabs present and the number of crabs observed
def calculate_nv_crabs_with_debugging(df):
    complete_data = pd.DataFrame() 

    grouped = df.groupby(['video file', 'tide category', 'selected observation period start'])

    for (video_file, tide_category, obs_start), group in grouped:
        present_population = group['present population'].iloc[0]
        present_males = group['present males'].iloc[0]
        present_females = group['present females'].iloc[0]

        observed_males = group.loc[group['sex'] == 'm', 'crab ID'].nunique()
        observed_females = group.loc[group['sex'] == 'f', 'crab ID'].nunique()

        num_nv_males = max(0, present_males - observed_males)
        num_nv_females = max(0, present_females - observed_females)

        print(f"Processing: {video_file}, {tide_category}, {obs_start}")
        print(f"Expected NV males: {num_nv_males}, Expected NV females: {num_nv_females}")

        # Check if adding the NV data will result in the expected row count
        nv_rows = pd.DataFrame()  

        for i in range(1, num_nv_males + 1):
            for minute in range(1, 31):
                nv_male_row = {
                    'video file': video_file,
                    'crabitat': group['crabitat'].iloc[0],
                    'season': group['season'].iloc[0],
                    'day type': group['day type'].iloc[0],
                    'tide category': tide_category,
                    'tide type': group['tide type'].iloc[0],
                    'present population': present_population,
                    'present sex ratio': group['present sex ratio'].iloc[0],
                    'present males': present_males,
                    'present females': present_females,
                    'selected observation period start': obs_start,
                    'real time': group['real time'].iloc[0],
                    'observation minute from start': minute,
                    'crab ID': f'NV_m{i}',
                    'sex': 'm',
                    'instantaneous behaviour': 'NV',
                    'human visible?': 'N'
                }
                nv_rows = pd.concat([nv_rows, pd.DataFrame([nv_male_row])], ignore_index=True)

        for i in range(1, num_nv_females + 1):
            for minute in range(1, 31):
                nv_female_row = {
                    'video file': video_file,
                    'crabitat': group['crabitat'].iloc[0],
                    'season': group['season'].iloc[0],
                    'day type': group['day type'].iloc[0],
                    'tide category': tide_category,
                    'tide type': group['tide type'].iloc[0],
                    'present population': present_population,
                    'present sex ratio': group['present sex ratio'].iloc[0],
                    'present males': present_males,
                    'present females': present_females,
                    'selected observation period start': obs_start,
                    'real time': group['real time'].iloc[0],
                    'observation minute from start': minute,
                    'crab ID': f'NV_f{i}',
                    'sex': 'f',
                    'instantaneous behaviour': 'NV',
                    'human visible?': 'N'
                }
                nv_rows = pd.concat([nv_rows, pd.DataFrame([nv_female_row])], ignore_index=True)

        # Concatenate observed data and NV rows
        full_group_data = pd.concat([group, nv_rows], ignore_index=True)
        expected_rows = present_population * 30
        actual_rows = full_group_data.shape[0]

        print(f"Expected total rows for {video_file} {tide_category}: {expected_rows}, Actual: {actual_rows}")

        # Append only if row count matches expectation
        if actual_rows == expected_rows:
            complete_data = pd.concat([complete_data, full_group_data], ignore_index=True)
        else:
            print(f"Skipping {video_file} {tide_category} due to row mismatch.")

    column_order = ['video file', 'crabitat', 'season', 'day type', 'tide category', 'tide type', 'present population',
                    'present sex ratio', 'present males', 'present females', 'selected observation period start',
                    'real time', 'observation minute from start', 'crab ID', 'sex', 'instantaneous behaviour', 'human visible?']

    complete_data = complete_data[column_order]
    return complete_data

df_with_nv_debugged = calculate_nv_crabs_with_debugging(df_with_tide_categories_and_types)

# output_path_2 = f"{file_name}+NV{file_extension}" # note the identical instantaneous-behaviour-raw-data.xlsx and instantaneous-behaviour-raw-data.csv have already been through this step and possess the NV concatenation (however they do not possess +NV in the file name). 
# df_with_nv_debugged.to_excel(output_path_2, index=False)
print(f"Cleaned dataframe including NVs saved to: {output_path_2}")

####################################################################
# VALIDATION & DUPLICATE CHECK (following NV concatenation)
####################################################################

def validate_data(df_with_nv_debugged):
    validation_results = []
    duplicates_list = []

    grouped = df_with_nv_debugged.groupby(['video file', 'tide category'])

    for (video_file, tide_category), group in grouped:
        present_population = group['present population'].iloc[0]
        expected_rows = present_population * 30
        
        actual_rows = group.shape[0]

        missing_rows = expected_rows - actual_rows

        validation_results.append({
            'video file': video_file,
            'tide category': tide_category,
            'expected rows': expected_rows,
            'actual rows': actual_rows,
            'missing rows': missing_rows
        })

        duplicate_columns = ['selected observation period start', 'observation minute from start', 'crab ID']
        duplicates = group[group.duplicated(subset=duplicate_columns, keep=False)]  

        if not duplicates.empty:
            duplicates_list.append({
                'video file': video_file,
                'tide category': tide_category,
                'duplicates': duplicates.to_dict(orient='records') 
            })

        for column in ['video file', 'tide category', 'present population']:
            if group[column].isnull().any():
                print(f"NaN values found in column '{column}' for video file {video_file}, tide category {tide_category}.")

        if not ((group['observation minute from start'] >= 1).all() and (group['observation minute from start'] <= 30).all()):
            print(f"Observation minutes out of expected range (1-30) for video file {video_file}, tide category {tide_category}.")

    validation_results_df_with_nv = pd.DataFrame(validation_results)

    validation_results_df_with_nv = validation_results_df_with_nv[validation_results_df_with_nv['missing rows'] != 0]

    duplicates_df_with_nv = pd.DataFrame(duplicates_list)

    print(validation_results_df_with_nv)
    print("Duplicates found:")
    print(duplicates_df_with_nv)

    return validation_results_df_with_nv, duplicates_df_with_nv

validation_results_df_with_nv, duplicates_df_with_nv = validate_data(df_with_nv_debugged)

# validation_summary_path = f"{file_name}_cleaned+NV_validation.xlsx"  # note the identical instantaneous-behaviour-raw-data.xlsx and instantaneous-behaviour-raw-data.csv have already been through this step and possess 30 rows of observations for each crab, comprising into full observation windows (however they do not possess _validation in the file name). 
# duplicates_summary_path = f"{file_name}_cleaned+NV_duplicates.xlsx" # note the identical instantaneous-behaviour-raw-data.xlsx and instantaneous-behaviour-raw-data.csv have already been through this step and do not possess duplicates (however they do not possess _duplicates in the file name). 

if not validation_results_df_with_nv.empty:
    validation_results_df_with_nv.to_excel(validation_summary_path, index=False)
    print(f"Validation summary saved to: {validation_summary_path}")
else:
    no_issues_df = pd.DataFrame({'Message': ['No validation issues found.']})
    no_issues_df.to_excel(validation_summary_path, index=False)
    print("All crab IDs have the correct number of rows (30) in each observation window.")

if not duplicates_df_with_nv.empty:
    duplicates_df_with_nv.to_excel(duplicates_summary_path, index=False)
    print(f"Duplication summary saved to: {duplicates_summary_path}")
else:
    no_duplicates_df = pd.DataFrame({'Message': ['No duplicates found.']})
    no_duplicates_df.to_excel(duplicates_summary_path, index=False)
    print("No crab IDs possess duplicates.")